In [ ]:
import pandas as pd
from pathlib import Path

df = pd.read_csv("un-country-data.csv")
df


In [12]:
country_df = df[df["LocTypeName"] == "Country/Area"]
country_df.columns

Index(['SortOrder', 'LocID', 'Notes', 'ISO3_code', 'ISO2_code', 'SDMX_code',
       'LocTypeID', 'LocTypeName', 'ParentID', 'Location', 'VarID', 'Variant',
       'Time', 'TPopulation1Jan', 'TPopulation1July', 'TPopulationMale1July',
       'TPopulationFemale1July', 'PopDensity', 'PopSexRatio', 'MedianAgePop',
       'NatChange', 'NatChangeRT', 'PopChange', 'PopGrowthRate',
       'DoublingTime', 'Births', 'Births1519', 'CBR', 'TFR', 'NRR', 'MAC',
       'SRB', 'Deaths', 'DeathsMale', 'DeathsFemale', 'CDR', 'LEx', 'LExMale',
       'LExFemale', 'LE15', 'LE15Male', 'LE15Female', 'LE65', 'LE65Male',
       'LE65Female', 'LE80', 'LE80Male', 'LE80Female', 'InfantDeaths', 'IMR',
       'LBsurvivingAge1', 'Under5Deaths', 'Q5', 'Q0040', 'Q0040Male',
       'Q0040Female', 'Q0060', 'Q0060Male', 'Q0060Female', 'Q1550',
       'Q1550Male', 'Q1550Female', 'Q1560', 'Q1560Male', 'Q1560Female',
       'NetMigrations', 'CNMR'],
      dtype='object')

In [15]:
columns = ['ISO3_code', 'Location', '1', '7', 'Time']
new_df = country_df.rename(columns={"TPopulation1Jan": "1", "TPopulation1July": "7"})
new_df = new_df[columns]
new_df

,ISO3_code,Location,1,7,Time
44384,BDI,Burundi,2229.322,2254.938,1950
44385,BDI,Burundi,2280.554,2305.746,1951
44386,BDI,Burundi,2330.938,2355.804,1952
44387,BDI,Burundi,2380.670,2405.186,1953
44388,BDI,Burundi,2429.703,2454.586,1954
...,...,...,...,...,...
84355,WLF,Wallis and Futuna Islands,7.161,7.122,2097
84356,WLF,Wallis and Futuna Islands,7.084,7.043,2098
84357,WLF,Wallis and Futuna Islands,7.002,6.958,2099
84358,WLF,Wallis and Futuna Islands,6.915,6.878,2100


In [16]:
df = new_df.melt(id_vars=["ISO3_code", "Location", "Time"], var_name="Month", value_name="Population")
df['Month'] = df['Month'].astype(int)

In [17]:
df

,ISO3_code,Location,Time,Month,Population
0,BDI,Burundi,1950,1,2229.322
1,BDI,Burundi,1951,1,2280.554
2,BDI,Burundi,1952,1,2330.938
3,BDI,Burundi,1953,1,2380.670
4,BDI,Burundi,1954,1,2429.703
...,...,...,...,...,...
72043,WLF,Wallis and Futuna Islands,2097,7,7.122
72044,WLF,Wallis and Futuna Islands,2098,7,7.043
72045,WLF,Wallis and Futuna Islands,2099,7,6.958
72046,WLF,Wallis and Futuna Islands,2100,7,6.878


In [20]:
df.sort_values(by=["ISO3_code", "Time", "Month"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,ISO3_code,Location,Time,Month,Population
0,ABW,Aruba,1950,1,42.194
1,ABW,Aruba,1950,7,42.738
2,ABW,Aruba,1951,1,43.282
3,ABW,Aruba,1951,7,43.858
4,ABW,Aruba,1952,1,44.434
...,...,...,...,...,...
72043,ZWE,Zimbabwe,2099,7,37096.560
72044,ZWE,Zimbabwe,2100,1,37131.556
72045,ZWE,Zimbabwe,2100,7,37166.572
72046,ZWE,Zimbabwe,2101,1,37201.588


In [21]:
rows = df[['ISO3_code', 'Location', 'Time']].drop_duplicates()
rows

,ISO3_code,Location,Time
0,ABW,Aruba,1950
2,ABW,Aruba,1951
4,ABW,Aruba,1952
6,ABW,Aruba,1953
8,ABW,Aruba,1954
...,...,...,...
72038,ZWE,Zimbabwe,2097
72040,ZWE,Zimbabwe,2098
72042,ZWE,Zimbabwe,2099
72044,ZWE,Zimbabwe,2100


In [23]:
dicts = []
for index, row in rows.iterrows():
    for i in range(1, 13):
        if i != 1 and i != 7:
            if row['Time'] == 2101:
                continue
            else:
                dicts.append({"ISO3_code": row["ISO3_code"], "Location": row["Location"], "Time": row["Time"], "Month": i, "Population": None})
        
merged = pd.concat((df, pd.DataFrame(dicts)))

final_df = merged.sort_values(by=["ISO3_code", "Time", "Month"])
final_df = final_df[~((final_df['Time'] == 2101) & (final_df['Month'] == 7))]

final_df


C:\Users\ipekt\AppData\Local\Temp\ipykernel_30280\796176339.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged = pd.concat((df, pd.DataFrame(dicts)))


,ISO3_code,Location,Time,Month,Population
0,ABW,Aruba,1950,1,42.194
0,ABW,Aruba,1950,2,NaN
1,ABW,Aruba,1950,3,NaN
2,ABW,Aruba,1950,4,NaN
3,ABW,Aruba,1950,5,NaN
...,...,...,...,...,...
357866,ZWE,Zimbabwe,2100,9,NaN
357867,ZWE,Zimbabwe,2100,10,NaN
357868,ZWE,Zimbabwe,2100,11,NaN
357869,ZWE,Zimbabwe,2100,12,NaN


In [24]:
final_df['Population'] = final_df['Population'].interpolate()
final_df

,ISO3_code,Location,Time,Month,Population
0,ABW,Aruba,1950,1,42.194000
0,ABW,Aruba,1950,2,42.284667
1,ABW,Aruba,1950,3,42.375333
2,ABW,Aruba,1950,4,42.466000
3,ABW,Aruba,1950,5,42.556667
...,...,...,...,...,...
357866,ZWE,Zimbabwe,2100,9,37178.244000
357867,ZWE,Zimbabwe,2100,10,37184.080000
357868,ZWE,Zimbabwe,2100,11,37189.916000
357869,ZWE,Zimbabwe,2100,12,37195.752000
